## Mount Google Drive

In [ ]:
from google.colab import drive

# Make sure to unmount drive at mount point
drive.flush_and_unmount()
drive.mount('/content/drive')

# Research Start

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pytz
import pandas as pd

from datetime import datetime
from pathlib import Path
from scipy import stats

%matplotlib inline

### Creating IDX Equities Time Series

In [2]:
csv_combined = Path.home() / Path('Documents/data-ab/idx_exported_combined.csv') # csv file Path
# csv_combined = Path('/mnt/c/Users/nikki/Documents/data-ab/idx_exported_combined.csv') # WSL Path

# idx_combined = pd.read_csv(csv_combined, parse_dates={'Date' : [1]})

In [2]:
csv_loc = Path.home() / Path('Documents/data-ab/idx_exported_csv') # csv folder Path
# csv_loc = Path(r'/content/drive/Shared drives/algo-clenow/idx_exported_csv') # Google Colab Path
# csv_loc = Path('/mnt/c/Users/nikki/Documents/data-ab/idx_exported_csv') # WSL Path

# IDX stocks' ticker always have 4 characters
files = list(csv_loc.glob('????.csv'))

In [3]:
"""
Create a dictionary where the key is the ticker
and the value is a pandas dataframe of the OHLC time series
"""
data_idx = {}
for file in files:
    data_idx[file.stem] = pd.read_csv(file,
                                      index_col='Date',
                                      parse_dates=True)
    # data_idx[file.stem] = data_idx[file.stem].tz_localize(tz='Asia/Jakarta')

In [4]:
data_idx['ASII'].tail()

,Open,High,Low,Close,Volume
Date,,,,,
2021-07-22,4930.0,5025.0,4900.0,4980.0,36287300
2021-07-23,5000.0,5000.0,4910.0,4950.0,26847700
2021-07-26,4950.0,4970.0,4770.0,4770.0,43373400
2021-07-27,4770.0,4830.0,4730.0,4760.0,42635500
2021-07-28,4780.0,4790.0,4710.0,4710.0,24083000


### Stocks on the Move


In [5]:
def momentum_score(ts):
    """
    Input:  Price time series.
    Output: Annualized exponential regression slope, 
            multiplied by the R2
    """
    # Make a list of consecutive numbers
    x = np.arange(len(ts)) 
    # Get logs
    log_ts = np.log(ts) 
    # Calculate regression values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts)
    # Annualize percent
    annualized_slope = (np.power(np.exp(slope), 252) - 1) * 100
    #Adjust for fitness
    score = annualized_slope * (r_value ** 2)
    return score

In [6]:
def volatility(ts, period=24):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change
    """
    return ts.pct_change().rolling(period).std().iloc[-1]

In [7]:
# How many (series) candles back for momentum calculation?
momentum_window = 96

# Create an empty DataFrame to store score
momentum_table = pd.DataFrame(columns=['ticker', 'score', 'vola', 'inv_vola'])

# Eliminated stocks list
eliminated_stocks = pd.DataFrame(columns=['ticker', 'score', 'vola', 'reason'])

# How many (series) candles+1 back for std dev calculation?
vola_window = 24

# How many (series) candles back for EWMA calculation?
ma_period_fast = 32
ma_period_slow = 128

# Loop the dictionary and calculate the momentum_score, then append it to pandas
for ticker, timeseries in data_idx.items():
    momentum_series = timeseries['Close'].iloc[-momentum_window:]
    score = momentum_score(momentum_series)
    vola_series = timeseries['Close']
    vola = volatility(vola_series, vola_window) * 16
    median_volume = timeseries['Volume'].rolling(vola_window).median().iloc[-1]
    ma_fast = timeseries['Close'].rolling(ma_period_fast).mean().iloc[-1]
    ma_slow = timeseries['Close'].rolling(ma_period_slow).mean().iloc[-1]

#     ewma = timeseries['Close'].ewm(span=ewma_period).mean().iloc[-1]

    # Need the stocks to exist at least 3 years prior (756 trading days)
    if len(timeseries) < 756:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'umur belum 3 tahun'},
                                                     ignore_index=True)
        continue
    
    # If median volume falls below 100k in the stocks, drop it 
    if median_volume < 100000:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'avg volume di bawah 100k'},
                                                     ignore_index=True)
        continue
        
    # If it has been suspended (daily vol == 0) more than once, drop it
    if timeseries['Volume'].iloc[-momentum_window:].tolist().count(0) > 1:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'pernah disuspend lebih dari 1x'},
                                                     ignore_index=True)
        continue
    
    momentum_table = momentum_table.append({'ticker': ticker,
                                            'score': score,
                                            'vola': vola,
                                            'median_vol': median_volume,
                                            'ma_fast': ma_fast,
                                            'ma_slow': ma_slow},
                                           ignore_index=True)
    
    momentum_table['inv_vola'] = 1 / momentum_table['vola']

In [8]:
print(f'Ada {len(momentum_table)} saham lolos')
print(f'Ada {len(eliminated_stocks)} saham tereliminasi')

Ada 307 saham lolos
Ada 423 saham tereliminasi


In [9]:
momentum_table.sort_values('score', ascending=False)[:50].to_clipboard()

In [10]:
momentum_table.sort_values('score', ascending=False)[:50]

,ticker,score,vola,inv_vola,ma_fast,ma_slow,median_vol
166,LPLI,1952.319074,1.426088,0.701219,319.56250,184.367188,4.335000e+05
31,BAJA,1220.175400,0.956315,1.045681,333.75000,244.257812,4.371100e+06
224,PRIM,985.018563,1.625161,0.615324,376.15625,246.398438,8.425950e+06
255,SMDR,947.693694,0.778303,1.284847,583.59375,401.812500,7.533500e+06
278,TMAS,762.154630,1.392418,0.718175,289.06250,203.742188,8.939300e+06
57,BMSR,674.862326,0.894548,1.117883,198.37500,136.757812,1.689350e+06
177,MCAS,650.775895,0.550582,1.816261,7921.09375,5716.445312,4.077500e+05
191,MRAT,594.288320,0.764561,1.307940,383.06250,252.328125,1.469200e+06
116,HERO,588.032372,0.777812,1.285657,1585.15625,1148.945312,1.423500e+05
22,ARII,541.952250,1.106967,0.903369,296.37500,229.257812,4.059000e+05


In [11]:
eliminated_stocks[eliminated_stocks['reason'] == 'umur belum 3 tahun'].sort_values('score', ascending=False)

,ticker,score,vola,reason
109,DCII,29470.006344,8.610667e-08,umur belum 3 tahun
118,DMMX,23844.394842,5.324636e-01,umur belum 3 tahun
384,TFAS,6652.140124,4.977920e-01,umur belum 3 tahun
167,HDIT,6563.154369,1.252974e+00,umur belum 3 tahun
282,NFCX,5453.030792,6.492168e-01,umur belum 3 tahun
...,...,...,...,...
110,DEAL,-73.796628,5.977582e-01,umur belum 3 tahun
124,DUCK,-85.330336,8.992857e-01,umur belum 3 tahun
84,CBMF,-89.938252,7.834523e-01,umur belum 3 tahun
227,KOTA,-95.546417,1.240815e+00,umur belum 3 tahun


In [12]:
eliminated_stocks[eliminated_stocks['reason'] == 'pernah disuspend lebih dari 1x'].sort_values('score', ascending=False)

,ticker,score,vola,reason
265,MLPL,61878.210997,0.539081,pernah disuspend lebih dari 1x
35,BABP,7981.151556,0.888874,pernah disuspend lebih dari 1x
266,MLPT,6457.865468,0.877397,pernah disuspend lebih dari 1x
268,MPPA,5609.749921,0.499141,pernah disuspend lebih dari 1x
53,BINA,4676.555560,0.588291,pernah disuspend lebih dari 1x
41,BBHI,2550.791236,1.883436,pernah disuspend lebih dari 1x
421,ZBRA,2092.972874,0.407384,pernah disuspend lebih dari 1x
271,MSIN,975.376511,1.917520,pernah disuspend lebih dari 1x
215,KBLV,810.747036,1.190922,pernah disuspend lebih dari 1x
197,INTD,663.055877,0.442893,pernah disuspend lebih dari 1x


In [13]:
eliminated_stocks[eliminated_stocks['reason'] == 'avg volume di bawah 100k'].sort_values('score', ascending=False)

,ticker,score,vola,reason
225,KONI,3726.748860,1.172846,avg volume di bawah 100k
238,LMAS,2125.657043,0.565867,avg volume di bawah 100k
370,SUPR,461.327744,0.085536,avg volume di bawah 100k
290,OMRE,416.376545,1.533849,avg volume di bawah 100k
23,ARTA,396.134237,1.075977,avg volume di bawah 100k
...,...,...,...,...
365,SRIL,-56.895392,0.000000,avg volume di bawah 100k
28,ASMI,-68.301582,0.770503,avg volume di bawah 100k
208,JKON,-69.782138,0.218358,avg volume di bawah 100k
155,GIAA,-71.862572,0.000000,avg volume di bawah 100k
